In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import keras.backend as K
import matplotlib.pyplot as plt

from collections import deque
from keras import layers, models, metrics
from sklearn.linear_model import LinearRegression

In [2]:
DIR = "/kaggle/input/predict-ai-model-runtime/npz_all/npz/"
FILE_TYPE = 'xla'
DATA_CONFIGS = ['tile', 'layout']
DATA_GROUP = ['train', 'valid', 'test']
LAYOUT_SUBGROUPS = ['random', 'default']

def data_group_splitter(data):
    return [data[data['data_group'] == group] for group in DATA_GROUP]

def generate_data_files_df():
    df = pd.DataFrame(columns = ['id', 'file_path', 'data_group', 'config'])
    for curr_config in os.listdir(DIR):
                          
        data_dir = [os.path.join(DIR, curr_config, FILE_TYPE)]
        if curr_config == 'layout':
            data_dir = [os.path.join(d, subgroup) for d in data_dir for subgroup in LAYOUT_SUBGROUPS]
        for dd in data_dir:
            for group in DATA_GROUP:
                df = pd.concat([df,
                                pd.DataFrame([{'id': file.split('/')[-1].split('.')[0],
                                               'file_path': os.path.join(dd, group, file),
                                               'data_group': group,
                                               'config': curr_config} for file in os.listdir(os.path.join(dd, group))])])

    return [data_group_splitter(df[df['config'] == config]) for config in DATA_CONFIGS]

(train_tile_data, valid_tile_data, test_tile_data), (train_layout_data, valid_layout_data, test_layout_data) = generate_data_files_df()

In [3]:
def create_differentiable_model(
    input_features,
    hidden_units,
    dropout_rate
):
    input_layer = layers.Input((-1, input_features), activation = 'relu')
    x = layers.BatchNormalization()(input_layer)
    for i in range(len(hidden_units) - 1):
        x = layers.Dense(hidden_units[i], activation = 'relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(dropout_rate)(x)
    x = layers.Dense(hidden_units[-1], activation = 'relu')(x)
    return models.Model(input_layer, x)

class GraphConvLayer(layers.Layer):
    def __init__(
        self,
        input_features,
        hidden_units,
        dropout_rate
    ):
        self.message_fn = create_differentiable_model(input_features, hidden_units, dropout_rate)
        self.update_fn = create_differentiable_model(input_features, hidden_units, dropout_rate)
        
    def get_messages(self, nodes):
        return self.message_fn(nodes)
    
    def aggregate(self, indices, neighbour_messages, nodes):
        return tf.math.unsorted_segment_mean(
            neighbour_messages, indices, nodes.shape[0]
        )
    
    def update(self, nodes, aggregated_messages):
        h_v = tf.concat([nodes, aggregated_messages], axis = 1)
        h_v = self.update_fn(h_v)
        h_v = tf.nn.l2_normalize(h_v, axis = -1)
        return h_v
    
    def call(self, inputs):
        nodes, edges = inputs
        targets, sources = edges
        neighbours = tf.gather(nodes, targets)
        neighbour_messages = self.get_messages(neighbours)
        aggregated_messages = self.aggregate(targets, neighbour_messages, nodes)
        return self.update(nodes, aggregated_messages)

class GCN(models.Model):
    
    def __init__(
        self,
        graph,
        K,
        input_features,
        hidden_units,
        dropout_rate,
        *args,
        **kwargs
    ):
        super.__init__(*args, **kwargs)
        self.nodes, self.edges = graph
        self.K = K
        self.gcn_layers = [GraphConvLayer(input_features, hidden_units, droput_rate) for i in range(K)]
        
    def call(self, input_nodes):
        x = tf.gather(self.nodes, input_nodes)
        for layers in self.gcn_layers:
            x = layers((self.nodes, self.edges)) + x
        return tf.gather(x, input_nodes)    